## Image classification using MobileNet
In essence, this code creates a model that uses a pre-trained MobileNet network for feature extraction and then adds fully connected layers for classification. The fully connected layer is the part that will be further trained to adapt to the specific data of your classification problem, while the MobileNet layers remain unchanged and are used to extract features from the images. 

In [8]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt

from keras.layers import Flatten, Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator , img_to_array, load_img
from keras.applications.mobilenet import MobileNet, preprocess_input 
from keras.losses import categorical_crossentropy

In [9]:
# Create a MobileNet model with an input shape of 224x224 pixels and 3 color channels (RGB).
# The parameter include_top=False means that the fully connected classification layers are not included.
model = MobileNet(input_shape=(224, 224, 3), include_top=False)

# To prevent further training of the MobileNet layers, all layers in the model are set as non-trainable.
for layer in model.layers:
    layer.trainable = False

# Add a Flatten layer to convert the model's output into a one-dimensional vector.
x = Flatten()(model.output)

# Add a Dense layer with 7 units and a 'softmax' activation function for final classification.
x = Dense(units=7, activation='softmax')(x)

# Create a new model that combines the MobileNet architecture with the added layers for classification.
model = Model(model.input, x)


In [10]:
model.compile(optimizer="adam", loss=categorical_crossentropy, metrics=["accuracy"])

# Preparing data for training and testing
This code prepares data for training and testing an image classification model using a pre-trained MobileNet architecture.

Data Augmentation for Training: Training data is augmented with zooming, shearing, horizontal flipping, and pixel value rescaling, enhancing its diversity and quality.

Loading Training Data: The training data is loaded from a specified directory. Images are resized to 224x224 pixels, and batches of 32 images are used.

Data Augmentation for Testing: For testing data, only pixel value rescaling is applied.

Loading Testing Data: Testing data is loaded from the designated directory.

This approach leverages a pre-trained MobileNet network for feature extraction, with additional fully connected layers for classification. The fully connected layers are trained to adapt to the specific classification task, while the MobileNet layers remain unaltered to extract meaningful features from the images.

In [11]:
train_data_augumentation = ImageDataGenerator(zoom_range=0.2, shear_range=0.2, horizontal_flip=True, rescale=1./225)
train_data = train_data_augumentation.flow_from_directory(directory=r"C:\Users\Calin PC\Documents\emotion-detect-project\data\train", target_size=(224,224), batch_size=32)
train_data.class_indices
   

Found 28709 images belonging to 7 classes.


{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

In [12]:
test_data_augumentation = ImageDataGenerator(rescale = 1./225)
test_data = test_data_augumentation.flow_from_directory(directory=r"C:\Users\Calin PC\Documents\emotion-detect-project\data\test", target_size=(224,224), batch_size=32)
test_data.class_indices

Found 7178 images belonging to 7 classes.


{'angry': 0,
 'disgust': 1,
 'fear': 2,
 'happy': 3,
 'neutral': 4,
 'sad': 5,
 'surprise': 6}

# Early Stopping and Model Check Point

In [13]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor="val_loss", min_delta=0.01, patience=5, verbose=1)
mc = ModelCheckpoint(filepath="best_model.h5", monitor= 'val_accuracy', verbose= 1, save_best_only= True, mode = 'auto')
call_back = [es,mc]

In [14]:
hist = model.fit_generator(train_data, 
                           steps_per_epoch= 10, 
                           epochs= 30, 
                           validation_data= test_data, 
                           validation_steps= 8, 
                           callbacks=[es,mc])

C:\Users\Calin PC\AppData\Local\Temp\ipykernel_20648\1036444180.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(train_data,


Epoch 1/30
10/10 [==============================] - ETA: 0s - loss: 18.3756 - accuracy: 0.2469
Epoch 1: val_accuracy improved from -inf to 0.28906, saving model to best_model.h5
10/10 [==============================] - 21s 2s/step - loss: 18.3756 - accuracy: 0.2469 - val_loss: 11.4051 - val_accuracy: 0.2891
Epoch 2/30


c:\Users\Calin PC\Documents\emotion-detect-project\venv\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


10/10 [==============================] - ETA: 0s - loss: 11.1075 - accuracy: 0.2438
Epoch 2: val_accuracy improved from 0.28906 to 0.31250, saving model to best_model.h5
10/10 [==============================] - 18s 2s/step - loss: 11.1075 - accuracy: 0.2438 - val_loss: 7.2169 - val_accuracy: 0.3125
Epoch 3/30
10/10 [==============================] - ETA: 0s - loss: 7.6830 - accuracy: 0.3625
Epoch 3: val_accuracy did not improve from 0.31250
10/10 [==============================] - 18s 2s/step - loss: 7.6830 - accuracy: 0.3625 - val_loss: 8.3527 - val_accuracy: 0.2422
Epoch 4/30
10/10 [==============================] - ETA: 0s - loss: 6.6247 - accuracy: 0.3594
Epoch 4: val_accuracy improved from 0.31250 to 0.43359, saving model to best_model.h5
10/10 [==============================] - 18s 2s/step - loss: 6.6247 - accuracy: 0.3594 - val_loss: 5.4044 - val_accuracy: 0.4336
Epoch 5/30
10/10 [==============================] - ETA: 0s - loss: 5.6561 - accuracy: 0.3875
Epoch 5: val_accuracy i